# Organizar submissoes



## Dependências

In [10]:
# Python 3.x
import pandas as pd
import numpy as np
import os

import sys
# to import utils
sys.path.append('../')
import utils

In [11]:
def get_problem_letter(row):
    return row[0]

In [12]:
def get_answer_bin(row):
    if "NO" in row:
        return 0
    else:
        return 1

In [44]:
def get_team_name(row, team_names):
    if row is None:
        return row

    for name in team_names:
        if str(row) in name:
            return name
    return row

In [126]:
from string import ascii_uppercase

def calculate_penalty(df):
    PENALTY = 'Penalty'
    df[PENALTY] = None
    for u in set(df.User):
        penalty = {k: 0 for k in ascii_uppercase}
        done = {k: False for k in ascii_uppercase}
        accum = 0
        for e, iid in zip(df[df.User == u].values, df.loc[df.User == u].index):
            time = e[3]
            accepted = e[e.shape[0]-2]
            problem = e[4]
            if done[problem]:
                continue
            if accepted == 1:  # if solution was accepted, pane
                accum += time + penalty[problem]
                df.loc[iid, PENALTY] = accum
                penalty[problem] = 0
                done[problem] = True
            else:
                df.loc[iid, PENALTY] = accum
                penalty[problem] += 20

    return df

In [127]:
folder_path = os.path.join(utils.OUTPUT_PATH,'submissoes/')
scoreboard_path = os.path.join(utils.OUTPUT_PATH,'scoreboards/')
years = os.listdir(folder_path)

for year in years:
    if 'pre_processado' in year:
        continue
    path = os.path.join(folder_path, year)
    df = pd.read_csv(path, skiprows=1)
    team_names = pd.read_csv(os.path.join(scoreboard_path, year), skiprows=1).Name

    # Clean existing columns
    df['Problem'] = df['Problem'].apply(lambda row: get_problem_letter(row))
    df['AnswerBin'] = df['Answer'].apply(lambda row: get_answer_bin(row))
    # Add university to team name if possible
    df['User'] = df['User'].apply(lambda row: get_team_name(row, team_names))
    # Calculate penalty
    df = calculate_penalty(df)
    path_to_csv = os.path.join(folder_path, 'pre_processado_' + str(year))
    utils.save_as_csv(df, path_to_csv)